In [1]:
import pylab as plt
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table, join

In [2]:
import seaborn as sns

This notebook uses all the raw data from the CIGALE predictions and photoz catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [3]:
Sel_func=pymoc.MOC()
Sel_func.read('../../dmu4/dmu4_sm_HDF-N/data/holes_HDF-N_irac1_O16_20190201_WARNING-MADE-WITH-Lockman-SWIRE-PARAMS.fits')

## Read in CIGALE predictions catalogue

In [4]:
cigale=Table.read('../../dmu28/dmu28_HDF-N/data/HDF-N_cigale_optnir_extcor_20180427.fits')

In [5]:
cigale['id'].name = 'help_id'

In [6]:
cigale

help_id,ra,dec,ebv,redshift,stellarity,flag_gaia,mosaic_u,mosaic_u_err,acs_f435w,acs_f435w_err,suprime_b,suprime_b_err,gpc1_g,gpc1_g_err,suprime_v,suprime_v_err,acs_f606w,acs_f606w_err,suprime_r,suprime_r_err,gpc1_r,gpc1_r_err,gpc1_i,gpc1_i_err,suprime_ip,suprime_ip_err,acs_f775w,acs_f775w_err,acs_f814w,acs_f814w_err,suprime_zp,suprime_zp_err,quirc_hk,quirc_hk_err,gpc1_z,gpc1_z_err,acs_f850lp,acs_f850lp_err,gpc1_y,gpc1_y_err,wfc3_f105w,wfc3_f105w_err,wfc3_f125w,wfc3_f125w_err,wfc3_f140w,wfc3_f140w_err,wfc3_f160w,wfc3_f160w_err,wircam_ks,wircam_ks_err,moircs_k,moircs_k_err,irac_i1,irac_i1_err,irac_i2,irac_i2_err,irac_i3,irac_i3_err,irac_i4,irac_i4_err
,deg,deg,,,,,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy
bytes27,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
HELP_J123503.293+621515.419,188.76372150513802,62.254283169711755,0.01019150286931473,0.6422372,nan,0,0.00012457534372344202,3.766983093620655e-05,nan,nan,0.00037093159439027156,2.362835443703174e-05,nan,nan,0.0008715470771225467,5.2812627057636727e-05,nan,nan,0.002209210869665933,4.34792934626456e-05,nan,nan,nan,nan,0.004460767201978454,0.00011034388034600003,nan,nan,nan,nan,0.0056416573659377875,0.00011307409788823542,0.0223475494357809,0.014673044834089884,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.014309201378914945,0.00020868839481239017,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
HELP_J123503.919+621610.085,188.766329505138,62.26946816971174,0.010291831728041611,0.7803647,nan,0,0.003592188914552406,0.00013031578829229628,nan,nan,nan,nan,nan,nan,0.0030657231792039093,6.463528160905694e-05,nan,nan,0.004403813008012354,5.475395684901884e-05,nan,nan,nan,nan,0.005335020861874162,0.00012034645565669116,nan,nan,nan,nan,0.005011166592988484,0.00012616272851872612,0.014420450922091805,0.007836348179745056,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00820031601016766,0.0002488288719590823,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
HELP_J123504.655+621539.737,188.76939450513802,62.26103816971175,0.01021946589366344,0.6621503,nan,0,0.000707718939452209,6.326429813663715e-05,nan,nan,0.0010390800753708113,2.9285386720373577e-05,nan,nan,0.0022124743038865477,5.9238643117481034e-05,nan,nan,0.003632680160640546,5.048382448196435e-05,nan,nan,nan,nan,0.004644252560292363,0.00011150174004026988,nan,nan,nan,nan,0.005142753591953785,0.00011961509081538304,0.001978154080307473,0.007352380810842247,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.010480666996181663,0.0002688894079051011,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
HELP_J123505.022+621639.557,188.770924986992,62.27765484314524,0.010338930745740887,0.54832095,nan,0,0.0005960023800687729,8.106185487739004e-05,nan,nan,0.0019770724878836717,5.33174869615079e-05,0.0018946883502106125,0.0005510519846610432,0.005802537950243952,0.0001209704860384843,nan,nan,0.016934897137896397,0.00010774917050099337,nan,nan,nan,nan,0.03053827752897969,0.00022815783410045672,nan,nan,nan,nan,0.037692132722846106,0.00025481075441523434,0.1334424207175961,0.012852910794109005,nan,nan,nan,nan,0.03138491433955076,0.007820731776692985,nan,nan,nan,nan,nan,nan,nan,nan,0.12982041950914885,0.000311040839409489,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
HELP_J123505.064+621601.550,188.771101505138,62.26709716971175,0.010257400281080424,0.5835653,nan,0,0.0006546246741040712,6.054235227686613e-05,nan,nan,0.0009751433544652229,2.758802945402771e-

## Read in photoz

In [4]:
photoz=Table.read('../../dmu24/dmu24_HDF-N/data/')

In [5]:
photoz

help_id,RA,DEC,id,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,za_hb,chi_r_eazy,chi_r_atlas,chi_r_cosmos,chi_r_stellar,stellar_type
bytes27,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes6
HELP_J141604.525+522314.318,214.01885343855852,52.38731062037661,1,0.3287,0.2017,0.46,0.79,-99.0,-99.0,-99.0,-99.0,0.3107445158917243,0.00941402,0.005750465,0.038778125,1.16068175,k4i
HELP_J141625.630+515253.992,214.10679106334706,51.881664522183016,2,0.3341,0.1838,0.4776,0.797,-99.0,-99.0,-99.0,-99.0,0.3186207796877177,0.172925325,0.057914225,0.093599325,1.79632475,m2v
HELP_J141911.366+530320.518,214.79735831765808,53.05569947987607,3,0.3557,0.1874,0.5225,0.791,-99.0,-99.0,-99.0,-99.0,0.3465622852301831,0.0053131625,0.03812535,0.0809428,2.155046,k5iii
HELP_J141525.065+522526.822,213.85443844608528,52.42411731941239,4,0.4947,0.3958,0.6165,0.433,0.2238,0.1284,0.299,0.358,0.42542435154728503,0.024677515,0.13246265,0.1400433,5.7028675,wg8iii
HELP_J141943.168+523158.983,214.92986686469408,52.53305086665113,5,0.2834,0.1183,0.4426,0.797,-99.0,-99.0,-99.0,-99.0,0.2912590674014151,0.00083104725,0.1810521,0.123727875,3.5283325,rk0v
HELP_J141848.923+525921.139,214.70384569336562,52.98920527391837,6,0.3486,0.1803,0.5225,0.79,-99.0,-99.0,-99.0,-99.0,0.2990182431374302,0.0020962585,0.0013756125,0.0577491,0.91910475,wk3iii
HELP_J142111.698+522044.991,215.29874092601307,52.345830748462134,7,0.4852,0.2721,0.6607,0.794,-99.0,-99.0,-99.0,-99.0,0.5270935167224258,0.0397356,0.233541025,0.39683775,1.8079045,m3iii
HELP_J141322.317+522430.572,213.3429865295886,52.40849213212558,8,0.3965,0.1909,0.5877,0.793,-99.0,-99.0,-99.0,-99.0,0.42542435154728503,0.000403126,0.014958385,0.0261745,5.6493875,wk3iii
HELP_J141553.974+523158.605,213.97489187629904,52.53294582179002,9,0.4897,0.3385,0.6458,0.791,-99.0,-99.0,-99.0,-99.0,0.49093314336472166,0.0200424975,0.00545671,0.0926184,5.765695,m2i


## Join CIGALE and photoz tables

In [9]:
prior=join(cigale,photoz)

In [10]:
from astropy.cosmology import Planck15 as cosmo
from astropy import units as u
f_pred=prior['bayes.dust.luminosity']/(4*np.pi*cosmo.luminosity_distance(prior['z1_median']).to(u.cm))

KeyError: 'bayes.dust.luminosity'

In [10]:
prior=prior[np.isfinite(f_pred.value)][np.log10(f_pred.value[np.isfinite(f_pred.value)])>8.5]

In [11]:
prior['DEC'].name='Dec'

In [13]:
len(cigale)

4436690

## Read in Maps

In [14]:
pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/HDF-N_SPIRE250_v1.0.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/HDF-N_SPIRE350_v1.0.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/HDF-N_SPIRE500_v1.0.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [15]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist['PRIMARY'].header
im250hdu=hdulist['IMAGE'].header

im250=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim250=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist['IMAGE'].header)
pixsize250=3600.0*w_250.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist['PRIMARY'].header
im350hdu=hdulist['IMAGE'].header

im350=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim350=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist['IMAGE'].header)
pixsize350=3600.0*w_350.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist['PRIMARY'].header
im500hdu=hdulist['IMAGE'].header

im500=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim500=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist['IMAGE'].header)
pixsize500=3600.0*w_500.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()

In [16]:
## Set XID+ prior class

In [17]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior['RA'] ,prior['Dec'] ,'hatlas_sgp_Ldust_prediction_results.fits',ID=prior['help_id'] )#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(prior['RA'] ,prior['Dec'] ,'hatlas_sgp_Ldust_prediction_results.fits',ID=prior['help_id'] )
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(prior['RA'] ,prior['Dec'] ,'hatlas_sgp_Ldust_prediction_results.fits',ID=prior['help_id'] )
prior500.prior_bkg(-5.0,5)

In [18]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0,1] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1,1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2,1] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [19]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 18279 tiles required for input catalogue and 379 large tiles
writing total_bytes=5118646130...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 2147483648)... done.
writing bytes [2147483648, 3221225472)... done.
writing bytes [3221225472, 4294967296)... done.
writing bytes [4294967296, 5118646130)... done.


SystemExit: 

In [57]:
prior250.nsrc

1236678